In [ ]:
#! pip install tqdm multiprocess

In [ ]:
import numpy as np
import pandas as pd
from multiprocess import Pool

from functions import evaluate_ar, evaluate_multiplier_iid, evaluate_ma
from bstrapping.synthetic_time_series.moving_average import MovingAverage

In [ ]:
mean = 4  # mean of the time series
alpha = 0.1

# Names of the stochastic processes

# Dependence coefficients of the stochastic processes, i.e. of the moving average processes
dependence_coefficients = [
    np.array([0]),
    np.array([0.5]),
    np.array([0.5 ** i for i in range(1, 3)]),
    np.array([0.5 ** i for i in range(1, 11)]),
    np.array([0.5 ** i for i in range(1, 16)]),
    np.array([0.5 ** i for i in range(1, 21)]),
]

names_dependence_coefficients = [
    "iid",
    "MA(1)",
    "MA(2)",
    "MA(10)",
    "MA(25)",
    "MA(20)",
]

list_name_weights = ['AR',
                     'Multiplier',
                     'MA',
                     ]

In [ ]:
benchmark = []
means_of_variance = []
std_of_variance = []

In [ ]:
sample_size = 2000
runs = 10
index_dependence = 3

time_series = MovingAverage(mean=mean, parameters=dependence_coefficients[index_dependence])
print("Time series: ",names_dependence_coefficients[index_dependence], "\nSample size: ", sample_size)

In [ ]:
samples = [
    MovingAverage(mean=mean, parameters=dependence_coefficients[index_dependence]).generate_samples(sample_size)
    for _ in range(runs)]

In [ ]:
asy_var = MovingAverage(mean=mean, parameters=dependence_coefficients[index_dependence]).asymptotic_variance

## Benchmark bootstraps

In [ ]:
betas = [0.1,0.2,"optimal",0.5,0.9]

In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()
evaluations_multiplier_ar = p.map(lambda sample: evaluate_ar(sample, alpha=alpha,number_bootstrap_samples=250,beta=0.1), samples)

result = np.array(evaluations_multiplier_ar)
means_of_variance.append(np.mean(result.T[0])-asy_var)
std_of_variance.append(np.std(result.T[0]))

In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()
evaluations_multiplier_ar = p.map(lambda sample: evaluate_ar(sample, alpha=alpha,number_bootstrap_samples=250,beta=0.2), samples)

result = np.array(evaluations_multiplier_ar)
means_of_variance.append(np.mean(result.T[0])-asy_var)
std_of_variance.append(np.std(result.T[0]))

In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()
evaluations_multiplier_ar = p.map(lambda sample: evaluate_ar(sample, alpha=alpha,number_bootstrap_samples=250,beta=2**(1/2)-1), samples)

result = np.array(evaluations_multiplier_ar)
means_of_variance.append(np.mean(result.T[0])-asy_var)
std_of_variance.append(np.std(result.T[0]))

In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()
evaluations_multiplier_ar = p.map(lambda sample: evaluate_ar(sample, alpha=alpha,number_bootstrap_samples=250,beta=0.5), samples)

result = np.array(evaluations_multiplier_ar)
means_of_variance.append(np.mean(result.T[0])-asy_var)
std_of_variance.append(np.std(result.T[0]))

In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()
evaluations_multiplier_ar = p.map(lambda sample: evaluate_ar(sample, alpha=alpha,number_bootstrap_samples=250,beta=0.9), samples)

result = np.array(evaluations_multiplier_ar)
means_of_variance.append(np.mean(result.T[0])-asy_var)
std_of_variance.append(np.std(result.T[0]))

## Concatination result

In [ ]:
benchmark = pd.DataFrame([means_of_variance,std_of_variance],
                         columns = pd.MultiIndex.from_tuples([("Beta",beta) for beta in betas]),
                        index=["Difference to mean","Std"])

In [ ]:
benchmark

In [ ]:
#benchmark.to_csv(f"./data/benchmark_{sample_size}_{names_dependence_coefficients[index_dependence]}.csv")
#benchmark.to_pickle(f"./data/benchmark_{sample_size}_{names_dependence_coefficients[index_dependence]}.pkl")